<a href="https://colab.research.google.com/github/Faissen/Hypothesis-and-A-B-testing/blob/main/Hypothesis_testing_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare the data

In [3]:
# Import libraries
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import re
import numpy as np
from scipy import stats as st

In [4]:
#Import the data
hypothesis = pd.read_csv('/datasets/hypotheses_us.csv', sep=';')
print(hypothesis.head(), '\n')

# Check the data
print(hypothesis.info(), '\n')
print(hypothesis.describe(), '\n')
print(f'Duplicated lines: ', hypothesis.duplicated().sum(), '\n')
print(f'Null-values by columns: ', '\n', hypothesis.isnull().sum())


FileNotFoundError: [Errno 2] No such file or directory: '/datasets/hypotheses_us.csv'

No null values, nor duplicated lines. Data types are correct.

In [ ]:
# Snake_case
new_columns_name = []
for column_name in hypothesis.columns:
    name_lowered = column_name.lower()
    new_columns_name.append(name_lowered)

hypothesis.columns = new_columns_name
print(hypothesis.head())

# Part 1 - Prioritizing hypothesis

## 1 - Classification following framework ICE

In [ ]:
# ICE = Impact x Confidence x Ease
# ICE score = (Impact x Confidence) / Ease

hypothesis['ice'] = round(((hypothesis['impact'] * hypothesis['confidence']) / hypothesis['effort']),1)
hypothesis = hypothesis.sort_values(by='ice', ascending=False)
hypothesis_ice = hypothesis[['hypothesis','ice']]
print(hypothesis_ice)

## 2 - Classification following framework RICE

In [ ]:
# RICE = Reach x Impact x Confidence x Ease
# ICE score = (Reach x Impact x Confidence) / Ease

hypothesis['rice'] = round(((hypothesis['reach'] * hypothesis['impact'] * hypothesis['confidence']) / hypothesis['effort']),1)
hypothesis = hypothesis.sort_values(by='rice', ascending=False)
hypothesis_rice = hypothesis[['hypothesis','rice']]
print(hypothesis_rice)

## 3 - How does the classification change between ICE and RICE? Why does it happen?

The main difference between ICE and RICE is that RICE considers how many users will be impacted by the change.

The main hypohesis from ICE are hypothesis 8, 0, 7 and 6.

The main hypohesis from RICE are hypothesis 7, 2, 0 and 6.

Hypothesis 0 and 7 appears in both analysis in the first four places.

In [ ]:
print(f'Hypothesis 0: ', hypothesis['hypothesis'][0], '\n',f'Hypothesis 7: ', hypothesis['hypothesis'][7])

# Part 2 - A/B testing

In [ ]:
#Import the data (optimized option)
orders = pd.read_csv('/datasets/orders_us.csv', sep=',', dtype={'group':'category'} , parse_dates=['date'])
print(orders.head(), '\n')

# Check the data
print(orders.info(), '\n')
print(orders.describe(), '\n')
print(f'Duplicated lines: ', orders.duplicated().sum(), '\n')
print(f'Null-values by columns: ', '\n', orders.isnull().sum())

No null values, nor duplicated lines. Data types are correct fter loading group and date with the most adequate data types.

In [ ]:
# Snake_case
def snake_columns(df):
    df.columns = [
        re.sub(r'(?<!_)Id$', '_id', col)  # adds _ prior to 'Id' at the end
           .replace('Id', '_id')
           .lower()
        for col in df.columns
    ]
    return df

snake_columns(orders)
print(orders.head())

In [ ]:
#Import the data (optimized option)
visits = pd.read_csv('/datasets/visits_us.csv', sep=',', dtype={'group':'category'} , parse_dates=['date'])
print(visits.head(), '\n')

# Check the data
print(visits.info(), '\n')
print(visits.describe(), '\n')
print(f'Duplicated lines: ', visits.duplicated().sum(), '\n')
print(f'Null-values by columns: ', '\n', visits.isnull().sum())

No null values, nor duplicated lines. Data types are correct fter loading group and date with the most adequate data types.

## 1- Plot cumulated revenue by group. Take conclusions and build conjectures.

In [ ]:
revenue_by_group = orders.groupby('group')['revenue'].sum().reset_index()
#print(revenue_by_group)

plt.bar(revenue_by_group['group'], revenue_by_group['revenue'])
plt.title('Total revenue by group')
plt.xlabel('Group')
plt.ylabel('Revenue')
plt.show()

revenue_rate = round(revenue_by_group[revenue_by_group['group']=='B']['revenue'].values[0] /
                     revenue_by_group[revenue_by_group['group']=='A']['revenue'].values[0] ,1)

print(f'Revenue of group B is {revenue_rate}x bigger than revenue of group A.')

Why is group B total revenue bigger than group A? Do both groups have the same size and the same time period?

## 2- Plot average cumulated revenue by group. Take conclusions and build conjectures.

In [ ]:
revenue_by_group_and_date = orders.groupby(['date','group'])['revenue'].mean().reset_index()
revenue_by_group_and_date.rename(columns={'revenue':'avg_revenue'}, inplace=True)
print(revenue_by_group_and_date.head())

plt.figure(figsize=(12, 6))
sns.lineplot(data=revenue_by_group_and_date, x='date',y='avg_revenue', hue='group')
plt.title('Average cumulated revenue by group')
plt.xlabel('Date')
plt.ylabel('Average revenue')
plt.tight_layout()
plt.show()

There are some atypical vales in both groups B, more visible in group B.

## 3- Plot the relative difference of the average cumulated revenue of group B regarding group A.  Take conclusions and build conjectures.

In [ ]:
cumulative_revenueA = revenue_by_group_and_date[revenue_by_group_and_date['group']=='A'].rename(columns={'avg_revenue':'avg_revenueA'}).drop(columns='group')
cumulative_revenueB = revenue_by_group_and_date[revenue_by_group_and_date['group']=='B'].rename(columns={'avg_revenue':'avg_revenueB'}).drop(columns='group')
cumulative_revenue_merged = cumulative_revenueA.merge(cumulative_revenueB, on='date', how='left')
#print(cumulative_revenueA.head(), '\n')
#print(cumulative_revenueB.head(), '\n')
print(cumulative_revenue_merged.head(), '\n')

plt.plot(cumulative_revenue_merged['date'],(cumulative_revenue_merged['avg_revenueB'] / cumulative_revenue_merged['avg_revenueA'])-1)
plt.axhline(y=0, color='grey',linestyle='--')
plt.tight_layout()
plt.xticks(rotation=45)
plt.title('Average cumulated revenue by group')
plt.xlabel('Date')
plt.ylabel('Average cumulated revenue ratio (B/A -1)');

Trends with some variability and inconsistencies. May be related to atypical values detected previouslly. Nevetheless, from 28.August group A average revenue started increasing compared with group B.

## 4- Calculate the conversion rate of each group as the ratio of orders to the number of visits for each day. Plot the daily conversion rates of the two groups and describe the difference. Draw conclusions and formulate hypotheses.

In [ ]:
# Conversion rate = daily_orders / daily_visits
daily_orders = orders.groupby('date')['transaction_id'].nunique().reset_index()
print(daily_orders.head(),'\n')

daily_visits_by_group = visits.groupby(['date','group'])['visits'].sum().reset_index()
print(daily_visits_by_group.head(),'\n')

daily_visits_by_group_merged = daily_visits_by_group.merge(daily_orders,on='date',how='left')
#print(daily_visits_by_group_merged.head(),'\n')

daily_visits_by_group_merged['conversion_rate'] = round(daily_visits_by_group_merged['transaction_id'] / daily_visits_by_group_merged['visits'] *100,2)
print(daily_visits_by_group_merged.head(),'\n')

plt.figure(figsize=(12, 6))
sns.lineplot(data=daily_visits_by_group_merged, x='date',y='conversion_rate', hue='group')
plt.title('Average cumulated revenue by group')
plt.xlabel('Date')
plt.ylabel('Conversion rate (%)')
plt.tight_layout()
plt.show()

Conversion rate for both groups is very instable.

## 5- Plot the relative difference in cumulative conversion for group B compared to group A. Draw conclusions and formulate hypotheses.

In [ ]:
cumulative_conversionA = daily_visits_by_group_merged[daily_visits_by_group_merged['group']=='A'].rename(
    columns={'conversion_rate':'conversion_rateA'}).drop(columns=['group','visits','transaction_id'])
cumulative_conversionB = daily_visits_by_group_merged[daily_visits_by_group_merged['group']=='B'].rename(
    columns={'conversion_rate':'conversion_rateB'}).drop(columns=['group','visits','transaction_id'])
cumulative_conversion_merged = cumulative_conversionA.merge(cumulative_conversionB, on='date', how='left')
#print(cumulative_conversionA.head(), '\n')
#print(cumulative_conversionB.head(), '\n')
print(cumulative_conversion_merged.head(), '\n')


plt.plot(cumulative_conversion_merged['date'],
         (cumulative_conversion_merged['conversion_rateB'] / cumulative_conversion_merged['conversion_rateA'])
         -1)
plt.axhline(y=0, color='grey',linestyle='--')
plt.tight_layout()
plt.xticks(rotation=45)
plt.title('Average cumulated conversion rate by group')
plt.xlabel('Date')
plt.ylabel('Average cumulated conversion rate ratio (B/A -1)');

Trends with some variability. Group B started better,than conversion rate started being below group A. Crrently conversion rate of group B is improving.

<div class="alert alert-block alert-success">
<b> Comentário do revisor: </b> <a class="tocSkip"></a>

- [x] A receita cumulativa por grupo foi analisada
- [x] A diferença relativa do tamanho médio do pedido foi calculada
- [x] A conversão por grupo foi analisada
- [x] A conclusão foi apresentada

</div>

## 6- Calculate the 95th and 99th percentiles for the number of orders per user. Define the point at which a data point becomes an anomaly.

In [ ]:
#Number of orders by user
orders_by_users = orders.groupby('visitor_id')['transaction_id'].nunique().reset_index().sort_values(by='transaction_id', ascending=True)
print(orders_by_users.head(),'\n')

percentiles = np.percentile(orders_by_users['transaction_id'],[95,99])
print(f'95% of users made {percentiles[0]} orders or less')
print(f'99% of users made {percentiles[1]} orders or less', '\n')

More than 2 orders per user may be considered an anomaly.

## 7- Create a scatter plot of the order prices. Draw conclusions and formulate hypotheses.

In [ ]:
#Number of orders by user
revenue_by_orders = orders.groupby('transaction_id')['revenue'].sum().reset_index()
print(revenue_by_orders.head(),'\n')

x_values = pd.Series(range(0,len(revenue_by_orders['revenue'])))
plt.scatter(revenue_by_orders['transaction_id'],revenue_by_orders['revenue'])
plt.tight_layout()
plt.xticks(rotation=45)
plt.title('Revenue by orders')
plt.ylabel('Revenue');

There are some atypical results that should be disregarded to obtain a more valid analysis.

## 8- Calculate the 95th and 99th percentiles of the order prices. Define the point at which a data point becomes an anomaly.

In [ ]:
percentiles2 = np.percentile(revenue_by_orders['revenue'],[95,99])
print(f'95% of orders is <= {percentiles2[0]}')
print(f'99% of orders is <= {percentiles2[1]}', '\n')

## 9- Find the statistical significance of the difference in conversion between the groups using the raw data. Draw conclusions and formulate hypotheses.

In [ ]:
print(cumulative_conversion_merged.head(), '\n')
# Print p-value
p_value = round(st.mannwhitneyu(cumulative_conversion_merged['conversion_rateA'], cumulative_conversion_merged['conversion_rateB'])[1],3)
print(f'p-value is {p_value}')

conversion_gain = cumulative_conversion_merged['conversion_rateB'].mean() / cumulative_conversion_merged['conversion_rateA'].mean() -1
print(f'Conversion gain from group B to group A is {round(conversion_gain*100,1)}%')

p-value is > 0.05 therefore we can not reject the null hypothesis. There is no statiscal significance regarding conversion between groups A and B.

## 10- Find the statistical significance of the difference in average order size between the groups using the raw data. Draw conclusions and formulate hypotheses.

In [ ]:
print(cumulative_revenue_merged.head(), '\n')
# Print p-value
p_value = round(st.mannwhitneyu(cumulative_revenue_merged['avg_revenueA'], cumulative_revenue_merged['avg_revenueB'])[1],2)
print(f'p-value is {p_value}')

revenue_gain = cumulative_revenue_merged['avg_revenueB'].mean() / cumulative_revenue_merged['avg_revenueA'].mean() -1
print(f'Revenue gain from group B to group A is {round(revenue_gain*100,1)}%')

p-value is > 0.05 therefore we can not reject the null hypothesis. There is no statiscal significance in revenue change between groups A and B even though the difference is~24%.

## 11- Find the statistical significance of the difference in conversion between the groups using the filtered data. Draw conclusions and formulate hypotheses.

In [ ]:
# Filter data

#95 and 99º percentile:
# Orders: 2, 4
# Revenue: 435.54, 900.90

orders_visits = orders.merge(visits,on=['date','group'], how='left')
#print(orders_visits.head())
#print(orders_visits.info())
orders_visits_filtered = orders_visits[
    (orders_visits['revenue'] <= 500) &
    (orders_visits.groupby('visitor_id')['transaction_id'].transform('count') < 4)
]

print(orders_visits_filtered.head())
#print(orders_visits_filtered.info())

# Conversion rate = daily_orders / daily_visits
daily_orders_filtered = orders_visits_filtered.groupby('date')['transaction_id'].nunique().reset_index()
print(daily_orders_filtered.head())

daily_visits_by_group_filtered = orders_visits_filtered.groupby(['date','group'])['visits'].sum().reset_index()
print(daily_visits_by_group_filtered.head(),'\n')

daily_visits_by_group_filtered_merged = daily_visits_by_group_filtered.merge(daily_orders_filtered,on='date',how='left')
print(daily_visits_by_group_filtered_merged.head(),'\n')

daily_visits_by_group_filtered_merged['conversion_rate'] = round(daily_visits_by_group_filtered_merged['transaction_id'] /
                                                                 daily_visits_by_group_filtered_merged ['visits'] *100,2)
print(daily_visits_by_group_filtered_merged.head(),'\n')


cumulative_conversionA_filtered = daily_visits_by_group_filtered_merged[daily_visits_by_group_filtered_merged['group']=='A'].rename(
    columns={'conversion_rate':'conversion_rateA'}).drop(columns=['group','visits','transaction_id'])
cumulative_conversionB_filtered = daily_visits_by_group_filtered_merged[daily_visits_by_group_filtered_merged['group']=='B'].rename(
    columns={'conversion_rate':'conversion_rateB'}).drop(columns=['group','visits','transaction_id'])
cumulative_conversion_merged_filtered = cumulative_conversionA_filtered.merge(cumulative_conversionB_filtered, on='date', how='left')
#print(cumulative_conversionA.head(), '\n')
#print(cumulative_conversionB.head(), '\n')
print(cumulative_conversion_merged_filtered.head(), '\n')

# Print p-value
p_value_filtered = round(st.mannwhitneyu(cumulative_conversion_merged_filtered['conversion_rateA'],
                                cumulative_conversion_merged_filtered['conversion_rateB'])[1],3)
print(f'p-value is {p_value_filtered}')

conversion_gain_filtered = (cumulative_conversion_merged_filtered['conversion_rateB'].mean() /
                   cumulative_conversion_merged_filtered['conversion_rateA'].mean()) -1
print(f'Conversion gain from group B to group A is {round(conversion_gain_filtered*100,1)}%')

p-value is < 0.05 therefore we can reject the null hypothesis. There is statiscal significance regarding conversion between groups A and B.

Group B presented significantly lower conversions than group A.

## 12- Find the statistical significance of the difference in average order size between the groups using the filtered data. Draw conclusions and formulate hypotheses.

In [ ]:
print(orders_visits_filtered.head())
revenue_by_group_and_date_filtered = orders_visits_filtered.groupby(['date','group'])['revenue'].mean().reset_index()
revenue_by_group_and_date_filtered.rename(columns={'revenue':'avg_revenue'}, inplace=True)
print(revenue_by_group_and_date.head())
cumulative_revenueA_filtered = revenue_by_group_and_date_filtered[
    revenue_by_group_and_date_filtered['group']=='A'].rename(columns={'avg_revenue':'avg_revenueA'}).drop(columns='group')
cumulative_revenueB_filtered = revenue_by_group_and_date_filtered[
    revenue_by_group_and_date_filtered['group']=='B'].rename(columns={'avg_revenue':'avg_revenueB'}).drop(columns='group')

cumulative_revenue_merged_filtered = cumulative_revenueA_filtered.merge(cumulative_revenueB_filtered, on='date', how='left')
print(cumulative_revenueA_filtered.head(), '\n')
print(cumulative_revenueB_filtered.head(), '\n')
print(cumulative_revenue_merged_filtered.head(), '\n')

# Print p-value
p_value_filt = round(st.mannwhitneyu(cumulative_revenue_merged_filtered['avg_revenueA'], cumulative_revenue_merged_filtered['avg_revenueB'])[1],2)
print(f'p-value is {p_value_filt}')

revenue_gain_filt = cumulative_revenue_merged_filtered['avg_revenueB'].mean() / cumulative_revenue_merged_filtered['avg_revenueA'].mean() -1
print(f'Revenue gain from group B to group A is {round(revenue_gain_filt*100,1)}%')

p-value is > 0.05 therefore we can not reject the null hypothesis. There is no statiscal significance in revenue change between groups A and B which makes sense since the difference is less than 2%.

## 13- Conclusions

Based on tests results after removing abnormal users:
* Conversion - There is statiscal significance regarding conversion between groups A and B. Group B presented significantly lower (~17%) conversions than group A.
* Average revenue - There is no statiscal significance in revenue change between groups A and B which makes sense since the difference is less than 2%.

We can terminate the test considering group A as leader.